In [9]:
import cv2
import numpy as np
import uuid

def resize(image, width=None, height=None, show=False):
   
    if width is None and height is None:
        return image

    if width is None:
        r = height / image.shape[0]
        width = int(r * image.shape[1])
    elif height is None:
        r = width / image.shape[1]
        height = int(r * image.shape[0])

    resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    if show:
        cv2.imshow('Image Resized By {0:.2f}'.format(r), resized)
        cv2.waitKey(0)

    return resized


def draw_corners(img, points):
    points=np.int_(points)
    if len(points) == 4:  # remarque hassan deplecte le code # solution usign random for this point 
        p1= points[0]
        p2= points[1]
        p3= points[2]
        p4 = points[3]
        cv2.circle(img, tuple(p1), 6, (0, 0, 255), -1)
        cv2.circle(img, tuple(p2), 6, (0, 255, 0), -1)
        cv2.circle(img, tuple(p3), 6, (255, 0, 0), -1)
        cv2.circle(img, tuple(p4), 6, (0, 255, 255), -1)

    else:
        from random import sample
        for p in points:
            
            random_color = tuple(sample(range(0, 255), 3))
            cv2.circle(img, tuple(p), 6, random_color, -1)

    cv2.imshow("image", img)
    cv2.waitKey(0)


def get_corner_points(contour):
    
    peri = cv2.arcLength(contour, True)
    
    corners = cv2.approxPolyDP(contour, 0.02 * peri, True)

    return np.squeeze(corners)


def order_corner_points_clockwise(points):
    
    rect = np.zeros((4, 2), dtype="float32")

    
    axis_sum = np.sum(points, axis=1)
    rect[0] = points[np.argmin(axis_sum)]
    rect[2] = points[np.argmax(axis_sum)]

   
    axis_diff = np.diff(points, axis=1)
    rect[1] = points[np.argmin(axis_diff)]
    rect[3] = points[np.argmax(axis_diff)]

  
    return rect


def apply_top_view(image, pts):
    (tl, tr, br, bl) = pts

    
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

   
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))


    dst = np.array([[0, 0],[maxWidth - 1, 0],[maxWidth - 1, maxHeight - 1],[0, maxHeight - 1]], dtype="float32")

   
    M = cv2.getPerspectiveTransform(pts, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

  
    return warped


def make_page_upright(src):

    image = None
    if isinstance(src, str): 
        image = cv2.imread(src)
    else:  
        image = src

    original_height, original_width = image.shape[:2]
    new_height = 400

   
    h_ratio = original_height / new_height

    
    small_image = resize(image, height=new_height)

    
    w_ratio = original_width / small_image.shape[1]

    gray_small_image = cv2.cvtColor(small_image, cv2.COLOR_BGR2GRAY)
    blurred_gray_small_image = cv2.GaussianBlur(gray_small_image, (9, 9), 0)
    canny_blurred_gray_small_image = cv2.Canny(
        blurred_gray_small_image, 120, 240)

    
    copy = small_image.copy()
    cnts = cv2.findContours(canny_blurred_gray_small_image,
                            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    biggest_contour = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

    
    unordered_corners = get_corner_points(biggest_contour)
   
    corners = order_corner_points_clockwise(unordered_corners)
    
    new_corners = np.zeros((4, 2), dtype="float32")
    for i in range(len(corners)):
        new_corners[i] = [corners[i][0] * w_ratio, corners[i][1] * h_ratio]

    return apply_top_view(image, new_corners)

In [34]:
image = cv2.imread("data/img2.jpeg")
# original demesion de image 
original_height, original_width = image.shape[:2]

new_height = 400
cv2.imshow("image", image)
cv2.waitKey(0)


# diffrence bteween original hight and new hieght
ratio = original_height / new_height

#resieze img 
small_image = resize(image, height=new_height)
cv2.imshow("image", small_image)
cv2.waitKey(0)

#change color to rgb format 
gray_small_image = cv2.cvtColor(small_image, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(gray_small_image, 150, 255,cv2.THRESH_BINARY)
cv2.imshow("image", thresh1)
cv2.waitKey(0)

#######################partie modifie  GaussianBlur(,(modifie,modifie),modifie) #############################
# cv2.GaussianBlur(src, ksize, sigmaX, sigmaY, borderType)
#blurred_gray_small_image = cv2.GaussianBlur(gray_small_image, (9,9),0 )
#cv2.imshow("image", blurred_gray_small_image)
#cv2.waitKey(0)

########### partie modfie canny( , modifie, modifie) ##############################


#canny_blurred_gray_small_image = cv2.Canny(blurred_gray_small_image, 130, 160)
#cv2.imshow("image", canny_blurred_gray_small_image)
#cv2.waitKey(0)
##########################################################

copy = small_image.copy()
cnts= cv2.findContours(thresh1,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

######################################## remarque hassan :
biggest_contour = sorted(cnts, key=cv2.contourArea, reverse=True)[0:2]  # le module statique extract 2 object beceause return list in tow index 
cv2.drawContours(copy, biggest_contour, -1, (0, 255, 0), 2)
cv2.imshow("image", copy)
cv2.waitKey(0)

for c in biggest_contour:
    unordered_corners = get_corner_points(c)
    print(len(unordered_corners))
    draw_corners(copy.copy(), unordered_corners)
            
    corners = order_corner_points_clockwise(unordered_corners)
    draw_corners(copy, corners)
            
    new_image = apply_top_view(image, np.float32(corners)*ratio)
    #new_image=cv2.rotate(new_image, cv2.ROTATE_180)
    #cv2.imwrite("./resulta/image"+str(uuid.uuid1())+".jpg",new_image)
    cv2.imshow("image", new_image)
    cv2.waitKey(0)
    cv2.imwrite("./resulta/ssssssss"+str(uuid.uuid1())+".jpg",new_image)
    
    #cv2.imwrite("hello1.jpg",im2)

4


In [38]:
import numpy as np
from PIL import Image
import cv2
import pytesseract

def is_array_or_pil(img):
    if isinstance(img, np.ndarray):
        return "Array"
    elif isinstance(img, Image.Image):
        return "PIL"
    else:
        raise TypeError("Input must be a NumPy array or PIL Image object")
def imgRotate(image,degree=0): 
    img=image   
    img=img if is_array_or_pil(img)=="Array" else np.array(img)

    def rotate_90(img):
        newimg=np.ones((img.shape[1],img.shape[0],img.shape[2]))
        for i in range(img.shape[0]-1):
            for j in range(img.shape[1]-1):
                newimg[j,i,:]=img[-i,j,:]
        return newimg
    
    def rotate_180(img):
        newimg=np.ones((img.shape[0],img.shape[1],img.shape[2]))
        newimg[:, :, :] = img[::-1,: :-1, :]
        return newimg
    
    def rotate_270(img):
        for i in range(3):
            img=rotate_90(img)
        return img
    
    if degree==-90 or degree==270:
        return rotate_270(img)
    elif abs(degree)==180:
        return rotate_180(img)
    elif degree==90 or degree==-270:
        return rotate_90(img)
    elif degree==0 :
        return img
    else:
        raise TypeError( str(degree) +"mod 90 != 0")

In [41]:
path='./resulta/33.jpg'
img = cv2.imread(path)

ret, thresh1 = cv2.threshold(img, 127, 255,cv2.THRESH_BINARY)
target=pytesseract.image_to_osd(thresh1,config='--psm 0 -c min_characters_to_try=5') # add character arabe for understand texte orientation 
degree= int(target.split("\n")[1].split(':')[1])
im2=imgRotate(img,degree=degree)
cv2.imwrite("./resulta/hello1.jpg",im2)


True

In [1]:
import cv2

In [ ]:
def resize_and_enhance_image(input_image_path, output_image_path, new_size, enhancement_factor):
    # Read the image
    img = cv2.imread(input_image_path)

    # Resize the image
    resized_img = cv2.resize(img, new_size)
    #resized_img = cv2.resize(cv2.imread(img , 2),new_size)

    # Enhance the image quality
    enhanced_img = cv2.detailEnhance(resized_img, sigma_s=10, sigma_r=0.95*enhancement_factor)

    # Save the enhanced image
    cv2.imwrite(output_image_path, enhanced_img)

# Example usage
input_image_path = 'im2.jpeg'
output_image_path = 'image.jpg'
new_size = (800, 600)  # Specify the desired dimensions (width, height)
enhancement_factor = 1.5  # Adjust the enhancement factor as needed

resize_and_enhance_image(input_image_path, output_image_path, new_size, enhancement_factor)
laplacian = cv2.Laplacian(img,cv2.CV_64F)
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)